# Modelo Vetorial  - Recuperação da Informação
**Aluno: Pedro Guedes Braga**


In [91]:
#coding: utf-8

import pandas as pd
import nltk
import operator
import math
import matplotlib.pyplot as pyplot
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import *
nltk.download('stopwords')
nltk.download('punkt') 
import csv
import time
import numpy
from operator import itemgetter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#**1 - Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice**




In [0]:
url_csv = "https://raw.githubusercontent.com/PedroGuedesBraga/lab06_ri/master/results.csv"
df = pd.read_csv(url_csv)
df.dropna(inplace=True)  ##Retira campos NA (ex.: NaN, etc)
lista_de_noticias = df.text.unique() #Retorna uma lista onde cada elemento é o texto de uma noticia.
 # http://www.nltk.org/howto/portuguese_en.html - Tutorial StopWords
 # https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/
palavras_a_ignorar = nltk.corpus.stopwords.words('portuguese')
palavras_a_ignorar.append('o')
palavras_a_ignorar.append('a')
palavras_a_ignorar.append('é')
palavras_a_ignorar.append('r')
  
special_chars = [',', '.', '-', '“', "”", ")", "(", ":", "%", "?", "$", "–", ""]



##Auxiliares
##Recebe um token(ja em lowercase) e aplica mais estratégias de tokenizacao, descritas pelas duas funcoes auxiliares acima.
def apply_tokenize_strategy(token):
  return remove_periods((remove_apostrophes(token)))

##Metodo auxiliar que remove apostrofos dos tokens, retornando um token em seu formato sem apostrofo
def remove_apostrophes(token):
  return token.replace("’", "").replace("'","")
  
##Metodo auxiliar que remove pontos em uma abreviacao
def remove_periods(token):
  return token.replace(".", "")

#Funcao auxiliar, que simplesmente resgata todos os documentos no indice
def getAllDocuments(index):
  list_of_documents = []
  for word in index:
    for tupla in index[word]:
      if(tupla[0] not in list_of_documents):
        list_of_documents.append(tupla[0])
        
  return list_of_documents







##Questao 1
##Funcao para construir o índice:
def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n = n + 1
    tokens = word_tokenize(document)
    for token in tokens:
      if((apply_tokenize_strategy(token.lower()) not in palavras_a_ignorar) and (apply_tokenize_strategy(token.lower()) not in special_chars) and ("," not in token.lower())):  ## Todos os tokens passam a serem tratados em LOWERCASE
        if(apply_tokenize_strategy(token.lower()) in index):
          index[apply_tokenize_strategy(token.lower())].append((n, tokens.count(token)))
        else:
          index[apply_tokenize_strategy(token.lower())] = [(n, tokens.count(token))]
  refactored_index = aux_refactor_index(index)
  return refactored_index


#funcao auxiliar responsavel por refatorar as listas invertidas do indice para retorna-la no formato pedido. - Remove duplicatas de documentos
def aux_refactor_index(index):
  for word in index:
    for tupla in index[word]:
      index[word] = list(index[word])
  return index
    
  
##Constrói o indice##
indice = build_index(lista_de_noticias)  








#**2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; (10 pts)**

In [93]:
indice = build_index(lista_de_noticias)
##Calcula o idf
number_of_documents = len(getAllDocuments(indice))
for word in indice:
  nk = len(indice[word])
  idf = numpy.log(number_of_documents / nk)
  indice[word] = list(indice[word]) #Converte set para lista - Só para melhor visualizacao mesmo
  indice[word].append(idf)


words = [word for word in indice][0:10]
doc_freq_list = [indice[word] for word in indice][0:10]
idfs = [indice[word][-1] for word in indice][0:10]
pd.DataFrame({
    'Palavras': words,
    'Doc freq list': doc_freq_list,
    'IDF': idfs
})

,Palavras,Doc freq list,IDF
0,juíza,"[(1, 2), (1, 2), (2, 1), 4.418840607796598]",4.418841
1,federal,"[(1, 1), (1, 1), (2, 1), (2, 1), (3, 1), (6, 1...",1.110734
2,ivani,"[(1, 1), (2, 1), 4.824305715904762]",4.824306
3,silva,"[(1, 3), (1, 3), (1, 3), (2, 1), (6, 1), (14, ...",2.878396
4,luz,"[(1, 3), (1, 3), (1, 3), (2, 1), (9, 1), (17, ...",2.472930
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4...",2.083466
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1), 4.1311585...",4.131159
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (...",3.214868
8,liminar,"[(1, 1), (2, 3), (2, 3), (2, 3), (119, 1), (21...",3.725693
9,nesta,"[(1, 2), (1, 2), (4, 1), (8, 1), (21, 1), (22,...",0.582979


#**3. Implementar as seguintes versões do modelo vetorial:(10 pts)**

## 3.1. Representação binária;

In [0]:
def binary(query, document):
  result = 0 #Score
  query_words = query.split()
  document_words = document.split()
  
  for word in query_words:
    result += (word in document_words)
  
  return result

## 3.2. TF (lembre-se que esse modelo já está implementado);

In [0]:
def term_frequency(query, document):
  result = 0
  query_words = query.split()
  document_words = document.split()
  
  for word in query_words:
    result += document_words.count(word)
  
  return result


## 3.3. TF-IDF

In [0]:
def tf_idf(query, document):
  result = 0
  document_words = document.split()
  query_words = query.split()
  
  for word in query_words:
    cwd = document_words.count(word)
    if word in indice:
      result += cwd * indice[word][-1]
      
  return round(result,2) #p nao ficar mt grande

## 3.4. BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho).

In [0]:
def bm25(query, document, k):
  result = 0
  doc_words = document.split()
  query_words = query.split()
  
  words = [word for word in query_words if word in doc_words]
    
  for word in words:
    cwd = doc_words.count(word)
    dfw = 0
    if word in indice:
      dfw = len(indice[word][:-1])
    result += (((k+1) * cwd) / (cwd + k)) * numpy.log10(((number_of_documents+1) / dfw)) if dfw != 0 else 0
  
  return round(result,2) #p n ficar mt grande no resultado

# 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;¶

In [98]:
consultas = ['jair bolsonaro', 'investigados policia', 'lava jato']


def create_top5_similar(query):
  n = 0 #doc number
  
  ##Listas de tuplas para cada algoritmo: [(score, doc_nmbr)]
  binary_docs = [] 
  tf_docs = []
  tf_idf_docs = []
  bm25_docs = []
  
  for doc in lista_de_noticias:
    doc = doc.lower()
    n = n + 1
    binary_docs.append((binary(query, doc),n))
    tf_docs.append((term_frequency(query, doc),n))
    tf_idf_docs.append((tf_idf(query, doc), n))
    bm25_docs.append((bm25(query, doc, 20), n))
  
  binary_docs.sort(reverse = True)
  tf_docs.sort(reverse=True)
  tf_idf_docs.sort(reverse=True)
  bm25_docs.sort(reverse=True)
    
  return (binary_docs[0:5], tf_docs[0:5], tf_idf_docs[0:5], bm25_docs[0:5])


top_5_first_query = create_top5_similar(consultas[0])
top_5_second_query = create_top5_similar(consultas[1])
top_5_third_query = create_top5_similar(consultas[2])

pd.DataFrame({
    'Consultas': consultas,
    'Binary': [top_5_first_query[0], top_5_second_query[0], top_5_third_query[0]],
    'Term Frequency': [top_5_first_query[1], top_5_second_query[1], top_5_third_query[1]],
    'TF_IDF': [top_5_first_query[2], top_5_second_query[2], top_5_third_query[2]],
    'BM25': [top_5_first_query[3], top_5_second_query[3], top_5_third_query[3]]
})


,Consultas,Binary,Term Frequency,TF_IDF,BM25
0,jair bolsonaro,"[(2, 238), (2, 237), (2, 231), (2, 224), (2, 2...","[(41, 207), (37, 151), (31, 166), (19, 19), (1...","[(11.18, 207), (1.99, 239), (1.99, 221), (1.99...","[(4.52, 207), (1.66, 151), (0.87, 239), (0.87,..."
1,investigados policia,"[(1, 235), (1, 207), (1, 206), (1, 204), (1, 1...","[(2, 207), (2, 206), (1, 235), (1, 204), (1, 1...","[(6.88, 207), (6.88, 206), (3.44, 235), (3.44,...","[(2.85, 207), (2.85, 206), (1.49, 235), (1.49,..."
2,lava jato,"[(2, 206), (2, 204), (2, 202), (2, 171), (2, 1...","[(6, 204), (5, 162), (3, 206), (2, 202), (2, 1...","[(18.2, 204), (15.16, 162), (9.1, 206), (6.07,...","[(7.22, 204), (6.13, 162), (3.84, 206), (2.64,..."


# 5. Compare os resultados encontrados e responda. (20 pts)

## 5.1 - Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

#### -> É razoável tomar a decisão de imediato excluir da análise os algoritmos de 'Representação Binária' e *'Term Frequency'*, pois se sabe que são mais simples e menos efetivos. Sendo assim, a análise fica entre o BM25 e o TF_IDF. Para esses dois modelos, observando os resultados para as consultas na tabela acima, percebe-se que apenas para a consulta "jair bolsonaro" os resultados diferem. Sendo assim, tomei a iniciativa de verificar mais a fundo esses documentos que diferem:

#### Documentos que diferem para query "jair bolsonaro": 

239 - "Atos por memória das vítimas da ditadura se multiplicam em revés a revisionismo de Bolsonaro"(TF_IDF) e 151 - "Grupos que sustentam Maduro serão principal desafio a qualquer futuro governo na Venezuela"(BM25) -

221- "Ferrovia Norte-Sul vale três vezes mais que previsto em edital  aponta estudo"(TF_IDF) e 239 - "Atos por memória das vítimas da ditadura se multiplicam em revés a revisionismo de Bolsonaro"(BM25)

#### A diferença entre os documentos recuperados em relação ao conteudo da consulta é muito sutil e dificulta um pouco para saber qual modelo trouxe melhores resultados. Porém, sendo necessário escolher um dos dois, escolho o BM25 como mais efetivo, pelo fato de que a notícia "Ferrovia Norte-Sul vale três vezes mais que previsto em edital  aponta estudo" pode talvez não ser diretamente envolvida com a query "Jair Bolsonaro"
